In [54]:
# !pip install openai -q
# !pip install langchain -q
# !pip install chromadb -q
# !pip install tiktoken -q
# !pip install pypdf -q
# !pip install unstructured[local-inference] -q
# !pip install gradio -q

In [50]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain

import os
os.environ["OPENAI_API_KEY"] = "Your API KEY"

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0,model_name="gpt-4")

In [31]:
import os

cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

Files in '/content/sample_data/Files to train': ['ADHD.pdf']


In [28]:
os.chdir("content/sample_data/Files to train/")

In [35]:
from langchain.document_loaders import DirectoryLoader

pdf_loader = DirectoryLoader('', glob="**/*.pdf")

loaders = [pdf_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Total number of documents: 1


In [36]:
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)

In [51]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

In [52]:
qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

In [53]:
import gradio as gr
# Define chat interface
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    chat_history = []

    def user(query, chat_history):
        print("User query:", query)
        print("Chat history:", chat_history)

        # Convert chat history to list of tuples
        chat_history_tuples = []
        for message in chat_history:
            chat_history_tuples.append((message[0], message[1]))

        # Get result from QA chain
        result = qa({"question": query, "chat_history": chat_history_tuples})

        # Append user message and response to chat history
        chat_history.append((query, result["answer"]))
        print("Updated chat history:", chat_history)

        return gr.update(value=""), chat_history


    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

User query: hi
Chat history: []
Updated chat history: [('hi', 'Hello! How can I assist you today?')]
User query: What is ADHD?
Chat history: [['hi', 'Hello! How can I assist you today?']]
Updated chat history: [['hi', 'Hello! How can I assist you today?'], ('What is ADHD?', 'Attention deficit hyperactivity disorder (ADHD) is a neurodevelopmental disorder. It is often recognized as a childhood disorder, but symptoms can persist into adulthood. It is common for adults with ADHD to experience high rates of co-morbidities, meaning they may have one or more additional psychiatric disorders. In some cases, these other disorders can overshadow the ADHD, causing it to be overlooked or undiagnosed.')]
Keyboard interruption in main thread... closing server.
